# df_brd_w_prd_v2에서 유실값 확인

In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [2]:
df_brd_w_prd_v1_drop_lg = pd.read_csv("df_brd_w_prd_v1_drop_lg.csv")
df_brd_w_prd_v1_drop_lg = df_brd_w_prd_v1_drop_lg.iloc[:, 1:]

In [3]:
def get_search_len(brd_name):

    try: 
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        print(userAgent)

        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument(f'user-agent={userAgent}')
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

        socket.setdefaulttimeout(30)

        url = 'https:/www.glowpick.com/searches/keywords'
        wd.get(url)
        time.sleep(5)
        
        html = wd.page_source
        soup = BeautifulSoup(html, "html.parser")

        wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
        wd.maximize_window()
        time.sleep(3)

        brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
        brd_input.send_keys(brd_name)
        brd_input.send_keys(Keys.ENTER)
        time.sleep(3)
        
        # ====

        #.products__product
        #.product
        # ====

        def parsing_html_with_class(parsing_info = '.products'):
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            interm_r = soup.select(parsing_info)
            return interm_r
        
        try:
            while True:

                product_lst = parsing_html_with_class()
                
                global product_len
                product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))
                

                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)

                product_lst_2 = parsing_html_with_class()

                if product_lst == product_lst_2:
                    html = wd.page_source
                    soup = BeautifulSoup(html, "html.parser")
                    prd_len = len(soup.find_all('div','products__product product'))
                    return prd_len
                    break


        except NoSuchElementException:
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            prd_len = len(soup.find_all('div','products__product product'))
            return prd_len
        
    except Exception as e:
        exception.append(brd_name)
        print("brand name: {}".format(brd_name), e)
        pass

In [5]:
exception = []

In [6]:
df_brd_w_prd_v1_drop_lg_0_10 = df_brd_w_prd_v1_drop_lg.loc[0:10]

In [7]:
for i in tqdm(list(df_brd_w_prd_v1_drop_lg_0_10.index)):
    brd_name = df_brd_w_prd_v1_drop_lg_0_10.loc[i, 'brand_name']
    prd_len = get_search_len(brd_name)
    df_brd_w_prd_v1_drop_lg_0_10.loc[i, 'prd_len'] = prd_len

  0%|          | 0/11 [00:00<?, ?it/s]



====== WebDriver manager ======


Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63]


====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


...

Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.2 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [8]:
df_brd_w_prd_v1_drop_lg_0_10

,brand_code_x,brand_name,product_link,product_name,brand_code_y,category,crawling_prd_cnt,prd_len
0,https://www.glowpick.com/brands/8016,가가알로에,['https://www.glowpick.com/products/104713'],['라율 나이트 케어 너리싱 젤'],https://www.glowpick.com/brands/8016,['스킨케어'],1.0,1.0
1,https://www.glowpick.com/brands/2077,가네보,"['https://www.glowpick.com/products/101993', '...","['페이스업 파우더 밀라노 콜렉션 2018 [SPF14/PA+]', '코프레도르 스...",https://www.glowpick.com/brands/2077,"['페이스메이크업', '립메이크업', '컨투어링', '클렌징', '배쓰&바디', '...",12.0,12.0
2,https://www.glowpick.com/brands/8724,가드미,"['https://www.glowpick.com/products/112824', '...","['큐티클 클리너', '네일 큐티클 호호바 오일', '[단종] 슈퍼 단백질 헤어가...",https://www.glowpick.com/brands/8724,['네일'],3.0,3.0
3,https://www.glowpick.com/brands/10352,가든 오브 라이프,"['https://www.glowpick.com/products/133823', '...","['비타민 로우 B 컴플렉스', '로우 프로바이오틱스 장케어', '무가공 비타민C'...",https://www.glowpick.com/brands/10352,['이너뷰티'],12.0,12.0
4,https://www.glowpick.com/brands/7314,가디스가든,"['https://www.glowpick.com/products/96142', 'h...","['유기농 드림 리페어 선 리페어 나이트 크림', '유기농 데이 언던 선 리페어 세...",https://www.glowpick.com/brands/7314,"['스킨케어', '베이비', '립메이크업', '클렌징', '선케어']",14.0,14.0
5,https://www.glowpick.com/brands/10037,가란시아,"['https://www.glowpick.com/products/129388', '...","['쉴퓨르즈 빠뜨 뒤 마라부', '쁘쉬 마쥐끄 누벨 뽀 비쟈쥬', '렐릭시르 뒤 마...",https://www.glowpick.com/brands/10037,"['스킨케어', '클렌징']",9.0,9.0
6,https://www.glowpick.com/brands/5601,가르니르,"['https://www.glowpick.com/products/74814', 'h...","['안나', '프라이드', '플란더스 EDT', '앨리스 EDT']",https://www.glowpick.com/brands/5601,['프래그런스'],4.0,4.0
7,https://www.glowpick.com/brands/4705,가르니에,"['https://www.glowpick.com/products/66001', 'h...","['미셀라 클렌징 워터', '프레시 아이 메이크업 리무버', '미셀라 클렌징 워터 ...",https://www.glowpick.com/brands/4705,"['스킨케어', '클렌징', '선케어', '배쓰&바디', '헤어']",22.0,22.0
8,https://www.glowpick.com/brands/9668,가오쿠,"['https://www.glowpick.com/products/124088', '...","['헤어 에센스', '헤어글로스 트리트먼트 샴푸', '헤어 마스크']",https://www.glowpick.com/brands/9668,['헤어'],3.0,3.0
9,https://www.glowpick.com/brands/10558,가온,"['https://www.glowpick.com/products/136313', '...","['새싹 담은 아이 & 페이스 크림', '씨앗담은 토닝 세럼', '[단종] 청순 ...",https://www.glowpick.com/brands/10558,['스킨케어'],18.0,18.0
